In [2]:
# pip install transformers
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 58.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=1c7a77c261960194a52a607ff38b45b10bc37c8e4680c736a59f42c86f3c5fc2
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [3]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00


In [4]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import sys, getopt
google_colab = True

if google_colab:
    root_path = "/content"
    train_arg = True
    startDate = '2021-01-01'
    endDate = '2021-04-01'
else:
    root_path = "."
    arguments = sys.argv[1:]
    short_options = "t:s:e:"
    long_options = ["train=", "startDate=", "endDate="]

    options, values = getopt.getopt(arguments, short_options, long_options)

    for o, v in options:
        print(f"Option is {o}. Value is {v}.")
        if o == "-t" or o == "--train":
            if v == "True":
                train_arg = True
            else:
                train_arg = False

        if o == "-s" or o == "--startDate":
            startDate = v
        if o == "-e" or o == "--endDate":
            endDate = v

import json
import torch
import pandas as pd
import datetime
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split

# from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, AdamW, get_scheduler
from torch.utils.data import DataLoader

from time import time

# TRAIN OR EVAL
train = train_arg
trained_model_name = "model_longformer.model"

# MODEL DEFINITION
if train:
    if google_colab:
        BASE_MODEL = "allenai/longformer-base-4096"
    else:
        BASE_MODEL = f"{root_path}/longformer-base-4096"
else:
    BASE_MODEL = "./" + trained_model_name
LEARNING_RATE = 2e-5
# BATCH_SIZE = 16
BATCH_SIZE = 1
EPOCHS = 20

# accelerator = Accelerator()

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)
optimizer = AdamW(model.parameters(), lr=3e-5)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# DATA PREPARATION

g_cases_filename = f"{root_path}/data/g_cases_2021.csv"
y = pd.read_csv(g_cases_filename)

X = pd.DataFrame()
# date_ranges = [['2021-01-01', '2021-04-01']]
# date_ranges = [['2020-01-01', '2020-01-12']]
date_ranges = [[startDate, endDate]]
for date_range in date_ranges:
    start = datetime.datetime.strptime(date_range[0], "%Y-%m-%d")
    end = datetime.datetime.strptime(date_range[1], "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

    for date in date_generated:
        date_str = date.strftime("%Y-%m-%d")

        # filename = f"./data/en_{date_str}_output.csv"
        filename = f"{root_path}/tokenize_longformer_output/tok_{date_str}_output.csv"
        df_data = pd.read_csv(filename)
        X = pd.concat([X, df_data], ignore_index=True)

print(f"X.shape: {X.shape}")
print(f"X.head(): {X.head()}")

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set, test size 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state=42, shuffle=False)
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=42, shuffle=False)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42, shuffle=False)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print("train_df")
print(f"train_df.shape: {train_df.shape}")
print(train_df)

print("val_df")
print(f"val_df.shape: {val_df.shape}")
print(val_df)

print("test_df")
print(f"test_df.shape: {test_df.shape}")
print(test_df)

raw_train_ds = Dataset.from_pandas(train_df)
raw_val_ds = Dataset.from_pandas(val_df)
raw_test_ds = Dataset.from_pandas(test_df)

ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

print("ds")
print(ds)

def preprocess_function(examples):
    label = examples["g_values"]
    print("examples['tokenized_tweets']")
    print(examples["tokenized_tweets"])
    s = examples["tokenized_tweets"].replace("\'", "\"")
    examples = json.loads(s)

    examples["label"] = float(label)
    return examples

from torch.utils.data import DataLoader

if train:
    ds["train"] = ds["train"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])
    ds["validation"] = ds["validation"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])

    train_dataloader = DataLoader(
        ds["train"], shuffle=False, batch_size=BATCH_SIZE, collate_fn=data_collator
    )
    eval_dataloader = DataLoader(
        ds["validation"], batch_size=BATCH_SIZE, collate_fn=data_collator
    )
else:
    ds["test"] = ds["test"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])

    test_dataloader = DataLoader(
        ds["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
    )

# for split in ds:
    # ds[split] = ds[split].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])

print("ds")
print(ds)

# for batch in train_dataloader:
#     batch = {k: v.to(device) for k, v in batch.items()}
#     break
# print({k: v.shape for k, v in batch.items()})

# outputs = model(**batch)
# print(outputs.loss, outputs.logits.shape)

def training():
    num_training_steps = EPOCHS * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    print(f"num_training_steps: {num_training_steps}")

    from tqdm.auto import tqdm

    progress_bar = tqdm(range(num_training_steps))

    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, batch in enumerate(train_dataloader):
            print(f"epoch: {epoch}, batch: {batch_idx}")
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            # accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

def evaluation(dataloader):
    eval_preds = []
    eval_labels = []
    model.eval()
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        print(f"outputs['logits'].shape: {outputs['logits'].shape}")
        eval_preds.append(outputs['logits'])

        print(f"batch['labels'].shape: {batch['labels'].shape}")
        eval_labels.append(batch['labels'])

    eval_preds = torch.cat(eval_preds, dim=0)
    print(f"eval_preds: {eval_preds}")
    eval_labels = torch.cat(eval_labels, dim=0)
    print(f"eval_labels: {eval_labels}")
    metrics = compute_metrics_for_regression(eval_preds, eval_labels)
    print(metrics)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred, eval_labels):
    print(f"eval_pred: {eval_pred}")
    print(f"eval_labels: {eval_labels}")
    logits = eval_pred
    labels = eval_labels
    labels = torch.reshape(labels, (-1, 1))
    print(f"labels: {labels}")

    mse = mean_squared_error(labels.cpu(), logits.cpu())
    mae = mean_absolute_error(labels.cpu(), logits.cpu())
    r2 = r2_score(labels.cpu(), logits.cpu())
    single_squared_errors = ((logits.cpu() - labels.cpu()).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

if train:
    ## TRAINING
    t = time()

    training()
    evaluation(eval_dataloader)
    model.save_pretrained(trained_model_name, from_pt=True)
    tokenizer.save_pretrained(trained_model_name)

    print('Time to train model: {} mins'.format(round((time() - t) / 60, 2)))

else:
    ## EVALUATION
    t = time()
    evaluation(test_dataloader)

    print('Time to eval model: {} mins'.format(round((time() - t) / 60, 2)))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkp

X.shape: (90, 2)
X.head():    created_at                                   tokenized_tweets
0  2021-01-01  {'input_ids': [0, 119, 506, 27833, 1595, 5548,...
1  2021-01-02  {'input_ids': [0, 705, 21103, 774, 471, 47268,...
2  2021-01-03  {'input_ids': [0, 17693, 30894, 22915, 4399, 2...
3  2021-01-04  {'input_ids': [0, 438, 1417, 808, 989, 1098, 2...
4  2021-01-05  {'input_ids': [0, 438, 1417, 808, 11883, 1161,...
train_df
train_df.shape: (62, 5)
    created_at                                   tokenized_tweets        date  \
0   2021-01-01  {'input_ids': [0, 119, 506, 27833, 1595, 5548,...  2021-01-01   
1   2021-01-02  {'input_ids': [0, 705, 21103, 774, 471, 47268,...  2021-01-02   
2   2021-01-03  {'input_ids': [0, 17693, 30894, 22915, 4399, 2...  2021-01-03   
3   2021-01-04  {'input_ids': [0, 438, 1417, 808, 989, 1098, 2...  2021-01-04   
4   2021-01-05  {'input_ids': [0, 438, 1417, 808, 11883, 1161,...  2021-01-05   
..         ...                                                ..

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

examples['tokenized_tweets']
{'input_ids': [0, 119, 506, 27833, 1595, 5548, 2792, 47268, 808, 21646, 4812, 1100, 32349, 4812, 1539, 47268, 808, 4022, 47268, 808, 23829, 21890, 25058, 1128, 14340, 9958, 47268, 808, 9937, 8020, 17835, 1220, 2003, 2391, 76, 15330, 28410, 179, 52, 548, 165, 1953, 645, 1576, 758, 340, 1218, 23681, 4783, 910, 4970, 179, 962, 47268, 808, 1528, 1528, 1034, 2851, 11914, 324, 910, 4970, 179, 35198, 12442, 2455, 1369, 6239, 2788, 9592, 47268, 808, 1759, 47268, 808, 1104, 790, 2026, 20125, 9479, 582, 1087, 582, 2105, 458, 7397, 629, 15828, 1623, 47320, 5262, 865, 310, 3524, 7521, 47268, 808, 7521, 265, 23387, 14414, 27828, 25916, 5015, 4478, 685, 633, 1425, 573, 4201, 9461, 18871, 28118, 2792, 35561, 323, 25241, 338, 5296, 539, 1522, 1372, 76, 47268, 808, 7321, 242, 5848, 1368, 35700, 1679, 3717, 5015, 2324, 1372, 10, 7399, 14712, 1098, 5968, 47268, 808, 1933, 1369, 66, 4349, 47268, 808, 8785, 621, 685, 633, 621, 962, 47268, 808, 20986, 47268, 808, 3997, 481, 1369

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

examples['tokenized_tweets']
{'input_ids': [0, 1397, 2407, 76, 17507, 2581, 82, 2579, 9240, 4488, 910, 13286, 47268, 808, 2841, 1916, 76, 9240, 4488, 6711, 2214, 2903, 13101, 15410, 284, 1098, 30218, 1626, 285, 47268, 808, 21911, 1227, 1035, 5627, 196, 15743, 5393, 2214, 7884, 9330, 1026, 47268, 808, 35075, 1407, 6229, 2205, 5770, 183, 3778, 4401, 1236, 7309, 919, 183, 1035, 6332, 22345, 10560, 22345, 825, 2311, 225, 8745, 26232, 1498, 4700, 34377, 1469, 19473, 3737, 2707, 3495, 36078, 5884, 15649, 2735, 3825, 2735, 1318, 898, 1787, 936, 47268, 808, 6556, 6680, 11433, 11445, 5401, 2097, 47268, 808, 28410, 7085, 493, 1956, 47268, 808, 403, 744, 28410, 7085, 493, 2504, 13134, 697, 343, 213, 26390, 47268, 808, 2269, 12074, 70, 1342, 3355, 1770, 31420, 12589, 884, 5807, 1765, 9937, 28404, 13969, 6428, 16934, 3551, 516, 3044, 931, 3087, 6072, 27534, 800, 10822, 4776, 1313, 47268, 808, 1956, 10409, 740, 34836, 35075, 34197, 1045, 4605, 47268, 808, 3186, 8701, 184, 27802, 317, 8921, 885, 4140

  0%|          | 0/1240 [00:00<?, ?it/s]

epoch: 0, batch: 0
epoch: 0, batch: 1


In [ ]:
import sys, getopt
google_colab = True

if google_colab:
    root_path = "/content"
    train_arg = False
    startDate = '2021-01-01'
    endDate = '2021-04-01'
else:
    root_path = "."
    arguments = sys.argv[1:]
    short_options = "t:s:e:"
    long_options = ["train=", "startDate=", "endDate="]

    options, values = getopt.getopt(arguments, short_options, long_options)

    for o, v in options:
        print(f"Option is {o}. Value is {v}.")
        if o == "-t" or o == "--train":
            if v == "True":
                train_arg = True
            else:
                train_arg = False

        if o == "-s" or o == "--startDate":
            startDate = v
        if o == "-e" or o == "--endDate":
            endDate = v

import json
import torch
import pandas as pd
import datetime
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader

from time import time

# TRAIN OR EVAL
train = train_arg
trained_model_name = "model_longformer_trainer.model"

# MODEL DEFINITION
if train:
    if google_colab:
        BASE_MODEL = "allenai/longformer-base-4096"
    else:
        BASE_MODEL = f"{root_path}/longformer-base-4096"
else:
    BASE_MODEL = "./" + trained_model_name
LEARNING_RATE = 2e-5
# BATCH_SIZE = 16
BATCH_SIZE = 1
EPOCHS = 20

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# DATA PREPARATION

g_cases_filename = f"{root_path}/data/g_cases_2021.csv"
y = pd.read_csv(g_cases_filename)

X = pd.DataFrame()
# date_ranges = [['2021-01-01', '2021-04-01']]
# date_ranges = [['2020-01-01', '2020-01-12']]
date_ranges = [[startDate, endDate]]
for date_range in date_ranges:
    start = datetime.datetime.strptime(date_range[0], "%Y-%m-%d")
    end = datetime.datetime.strptime(date_range[1], "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

    for date in date_generated:
        date_str = date.strftime("%Y-%m-%d")

        # filename = f"./data/en_{date_str}_output.csv"
        filename = f"{root_path}/tokenize_longformer_output/tok_{date_str}_output.csv"
        df_data = pd.read_csv(filename)
        X = pd.concat([X, df_data], ignore_index=True)

print(f"X.shape: {X.shape}")
print(f"X.head(): {X.head()}")

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set, test size 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state=42, shuffle=False)
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=42, shuffle=False)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42, shuffle=False)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print("train_df")
print(f"train_df.shape: {train_df.shape}")
print(train_df)

print("val_df")
print(f"val_df.shape: {val_df.shape}")
print(val_df)

print("test_df")
print(f"test_df.shape: {test_df.shape}")
print(test_df)

raw_train_ds = Dataset.from_pandas(train_df)
raw_val_ds = Dataset.from_pandas(val_df)
raw_test_ds = Dataset.from_pandas(test_df)

ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

print("ds")
print(ds)

def preprocess_function(examples):
    label = examples["g_values"]

    print("examples['tokenized_tweets']")
    print(examples["tokenized_tweets"])
    s = examples["tokenized_tweets"].replace("\'", "\"")
    examples = json.loads(s)


    examples["label"] = float(label)
    return examples

if train:
    ds["train"] = ds["train"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])
    ds["validation"] = ds["validation"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])
else:
    ds["test"] = ds["test"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])

print("ds")
print(ds)


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}



from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./models/longformer-fine-tuned-regression",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=10,
    save_total_limit = 5,
    # save_strategy="no",
    metric_for_best_model="mse",
    load_best_model_at_end=True,
    weight_decay=0.01,
)


import torch
from transformers import Trainer

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss


trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics_for_regression,
)


if train:
    ## TRAINING
    t = time()

    trainer.train()
    trainer.save_model(trained_model_name)
    tokenizer.save_pretrained(trained_model_name)

    print('Time to train model: {} mins'.format(round((time() - t) / 60, 2)))

else:
    ## EVALUATION
    t = time()

    trainer.eval_dataset=ds["test"]
    metrics = trainer.evaluate()
    print(metrics)

    print('Time to eval model: {} mins'.format(round((time() - t) / 60, 2)))

Didn't find file ./model_longformer_trainer.model/added_tokens.json. We won't load it.
loading file ./model_longformer_trainer.model/vocab.json
loading file ./model_longformer_trainer.model/merges.txt
loading file ./model_longformer_trainer.model/tokenizer.json
loading file None
loading file ./model_longformer_trainer.model/special_tokens_map.json
loading file ./model_longformer_trainer.model/tokenizer_config.json
loading configuration file ./model_longformer_trainer.model/config.json
Model config LongformerConfig {
  "_name_or_path": "./model_longformer_trainer.model",
  "architectures": [
    "LongformerForSequenceClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropo

X.shape: (90, 2)
X.head():    created_at                                   tokenized_tweets
0  2021-01-01  {'input_ids': [0, 119, 506, 27833, 1595, 5548,...
1  2021-01-02  {'input_ids': [0, 705, 21103, 774, 471, 47268,...
2  2021-01-03  {'input_ids': [0, 17693, 30894, 22915, 4399, 2...
3  2021-01-04  {'input_ids': [0, 438, 1417, 808, 989, 1098, 2...
4  2021-01-05  {'input_ids': [0, 438, 1417, 808, 11883, 1161,...
train_df
train_df.shape: (62, 5)
    created_at                                   tokenized_tweets        date  \
0   2021-01-01  {'input_ids': [0, 119, 506, 27833, 1595, 5548,...  2021-01-01   
1   2021-01-02  {'input_ids': [0, 705, 21103, 774, 471, 47268,...  2021-01-02   
2   2021-01-03  {'input_ids': [0, 17693, 30894, 22915, 4399, 2...  2021-01-03   
3   2021-01-04  {'input_ids': [0, 438, 1417, 808, 989, 1098, 2...  2021-01-04   
4   2021-01-05  {'input_ids': [0, 438, 1417, 808, 11883, 1161,...  2021-01-05   
..         ...                                                ..

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 14
  Batch size = 1


examples['tokenized_tweets']
{'input_ids': [0, 28781, 64, 2095, 694, 6461, 8218, 22481, 1296, 898, 792, 2524, 1532, 24255, 2865, 20125, 1161, 966, 24949, 394, 25584, 16812, 11573, 2675, 1447, 6789, 2927, 20884, 44203, 5853, 4560, 4362, 9055, 26459, 1329, 47268, 808, 1263, 82, 13295, 13078, 47268, 808, 27916, 4761, 554, 3151, 909, 82, 2416, 9937, 2035, 52, 548, 27344, 8581, 1104, 909, 33561, 3631, 542, 32019, 868, 1194, 36307, 282, 6527, 449, 21931, 102, 1437, 8649, 179, 909, 4026, 266, 47268, 808, 17301, 2237, 17960, 2988, 3299, 181, 6695, 821, 1517, 10302, 24668, 2806, 34377, 1469, 19473, 9937, 1632, 17381, 8785, 173, 4470, 5278, 3432, 24255, 2865, 681, 210, 671, 2340, 47268, 808, 14331, 39420, 102, 1077, 40067, 5312, 23387, 14414, 76, 434, 2635, 47268, 808, 9937, 1852, 3553, 46806, 6674, 4751, 1962, 6229, 31603, 1852, 11148, 139, 4342, 2794, 260, 8633, 4104, 8935, 1962, 3653, 10584, 47268, 808, 3044, 138, 23829, 459, 1296, 1467, 1040, 885, 2957, 260, 34377, 1469, 19473, 7967, 12769, 

Initializing global attention on CLS token...


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...


{'eval_loss': 0.1306784600019455, 'eval_mse': 0.1306784600019455, 'eval_mae': 0.35058069229125977, 'eval_r2': -2.5019359882323338, 'eval_accuracy': 1.0, 'eval_runtime': 6.2212, 'eval_samples_per_second': 2.25, 'eval_steps_per_second': 2.25}
Time to eval model: 0.1 mins
